<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---

# 랜덤포레스트(Random Forests)

### 1) 캐글 대회를 이어서 진행합니다. EDA, 데이터 전처리 부분을 업데이트 하세요.
- EDA는 항상 완벽할 수 없지요 한 주간 계속 같은 데이터로 과제를 진행하므로 부족한 부분을 추가하거나 논하세요.
- (지금은 feature engineering에 너무 시간을 들이지 마세요!)



In [1]:
### 이곳에서 과제를 진행해 주세요 ###

### 2) Ordinal Encoding을 적용해 보세요.


In [2]:
### 이곳에서 과제를 진행해 주세요 ###

### 3) (도전과제🔥) 수업에 사용하지 않은 다른 종류의 [category_encoders](http://contrib.scikit-learn.org/category_encoders/)을 적용해 보고 인코더에 대해 설명하세요



In [3]:
### 이곳에서 과제를 진행해 주세요 ###

### 4) 랜덤포레스트 모델을 적용한 후의 결과를 캐글에 제출하세요(랜덤포레스트를 적용하고 성능이 오히려 떨어졌을 수도 있습니다! 만약 그렇다면 왜 그럴까 고민을 해 보고 적어주세요(정답이 아니여도 괜찮습니다. 본인 논리로 고찰하세요))

In [4]:
### 이곳에서 과제를 진행해 주세요 ###

## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)